# Q2. Numerical Quadrature (Dr. Peterson)

## (a)
We are given the trapezoidal quadrature method below
\begin{equation*}
I(f) = \int_a^b f(x) dx = \dfrac{b-a}{2} \left[f(a) + f(b)\right] + K_1(b-a)^2 +
K_2(b-a)^4 + \dots
\end{equation*}
If we divide our domain $[a,b]$ using $M$ subintervals of length $h$, we can
evaluate the approximation $\hat{I}_h(f)$ for each subinterval and sum these
values over the entire domain to get $\hat{I}(f)$. The first error term is given
$\frac{f''(\xi)(b-a)}{12}(b-a)^2$ which we can use to evaluate the equispaced
trapezoid formula.
\begin{equation*}
\hat{I}(f) = \sum_{i=1}^{M} \left( \dfrac{h}{2} \left[f(a+h(i-1)) +
f(a+hi)\right] + \dfrac{f''(\xi)h^3}{12} \right)
\end{equation*}
We should get 2nd order accuracy for this method.

## b)
The approximation $\hat{I}(f)$ above is equivalent to $T_{M,1}$ for this
problem. To get $T_{2M,1}$ we sum over $2M$ intervals of length $h/2$.
\begin{equation*}
\begin{split}
T_{M,1} &= \sum_{i=1}^{M} \left( \dfrac{h}{2} \left[f(a+h(i-1)) + f(a+hi)\right]
+ \dfrac{f''(\xi)h^3}{12} + K_2h^4 \right) \\
&= \sum_{i=1}^{M} \dfrac{h}{2} \left[f(a+h(i-1)) + f(a+hi)\right] +
\dfrac{Mf''(\xi)h^3}{12} + MK_2h^4
\end{split}
\end{equation*}
\begin{equation*}
\begin{split}
T_{2M,1} &= \sum_{i=1}^{2M} \left( \dfrac{h}{4} \left[f(a+\frac{h(i-1)}{2}) +
f(a+\frac{hi}{2})\right] + \dfrac{f''(\xi)h^3}{96} + K_2h^4 \right) \\
&= \sum_{i=1}^{2M} \dfrac{h}{4} \left[f(a+\frac{h(i-1)}{2}) +
f(a+\frac{hi}{2})\right] + \dfrac{2Mf''(\xi)h^3}{96} + 2MK_2h^4
\end{split}
\end{equation*}
We can subtract $T_{M,1}$ from $T_{2M,1}$ in the following manner so that the
coefficients for both methods add up to 1.
\begin{equation*}
\begin{split}
T_{2M,2} &= \dfrac{4}{3} \left( T_{2M,1}-\dfrac{T_{M,1}}{4} \right) \\
&= \dfrac{4}{3} \sum_{i=1}^{2M} \dfrac{h}{4} \left[f(a+\frac{h(i-1)}{2}) +
f(a+\frac{hi}{2})\right] \\ &\hspace{1em} - \dfrac{1}{3} \sum_{i=1}^{M}
    \dfrac{h}{2} \left[f(a+h(i-1)) + f(a+hi)\right] + \dfrac{8MK_2h^4}{3} -
    \dfrac{MK_2h^4}{12}
\end{split}
\end{equation*}
We should get a 4th order accuracy for this combined method.

## c)

The following output shows the convergence rates for part a and b approaching 2 and 4 respectively:

In [9]:
import numpy as np

def f(x):
    f = np.sin(np.pi * x) # function
    return f
  
def exact():
    exact = 0.6366197723675814
    return exact
  
def trap(a, b, M):
    hs = np.linspace(a, b, M+1)
    h = hs[1] - hs[0]
    s = 0
    for i in range(hs.size-1):
        s += h/2 * ( f(hs[i]) + f(hs[i+1]) )
    return s
  
def rate(errors):
    rate = np.array([])
    for i in range(errors.size-1):
        rate = np.append(rate, np.log(errors[i] / errors[i+1]) / np.log(2))
    print rate
    return rate

### i.

In [7]:
a = 0
b = 1
Ms = np.array([1, 2, 4, 8, 16])
errors = np.array([])
for M in Ms:
    s = trap(a, b, M)
    errors = np.append(errors, np.abs(s - exact()))
rates = rate(errors)


[ 2.22026567  2.0467292   2.01125847  2.00278934]


### ii.

In [10]:
a = 0
b = 1
Ms = np.array([1, 2, 4, 8, 16])
Ms_2 = Ms[:] * 2
errors = np.array([])
for i in range(Ms_2.size):
    s1 = trap(a, b, Ms[i])
    s2 = trap(a, b, Ms_2[i])
    s = 4.0*(s2 - 1.0/4.0 * s1)/3.0
    errors = np.append(errors, np.abs(s - exact()))
rates = rate(errors)

[ 4.3716838   4.08236705  4.02004043  4.00497702]


## d)
We can use Romberg integration to combine approximations $T_{k,i}$ where i is
the new i'th `sequence`. In the pseudo code below, m represents the number of
step sizes.

    for all i in i's
      for all m in M's
        T_[m_next,i] = (4 ** (i-1) * T_[m_next] ** (i-1) - T_[m] ** (i-1)) / 4 ** (i-1) - 1
      end
    end
For this particular problem with $i=3$, this reduces to a program with one for loop.

    for all m in M's
      T_[m_next] = (4 ** 2 * T_[m_next] ** 2 - T_[m] ** 2) / 4 ** 2 - 1
    end